<table style="width:100%; background-color: #EBF5FB">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Time series: Processing Notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Processing Notebook</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/time_series">Time series Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory Notes](#1.-Introductory-Notes)
* [2. Settings](#2.-Settings)
	* [2.1 Import Python libraries](#2.1-Import-Python-libraries)
	* [2.2 Set version number and recent changes](#2.2-Set-version-number-and-recent-changes)
	* [2.3 Select timerange](#2.3-Select-timerange)
	* [2.4 Select download source](#2.4-Select-download-source)
	* [2.5 Select subset](#2.5-Select-subset)
* [3. Download](#3.-Download)
	* [3.1 Automatic download (for all sources except Energinet.dk)](#3.1-Automatic-download-%28for-all-sources-except-Energinet.dk%29)
	* [3.2 Manual download (Energinet.dk)](#3.2-Manual-download-%28Energinet.dk%29)
* [4. Read](#4.-Read)
	* [4.1 Preparations](#4.1-Preparations)
	* [4.2 Reading loop](#4.2-Reading-loop)
	* [4.3 Save raw data](#4.3-Save-raw-data)
* [5. Processing](#5.-Processing)
	* [5.1 Missing Data Handling](#5.1-Missing-Data-Handling)
	* [5.2 Country specific calculations](#5.2-Country-specific-calculations)
		* [5.2.1 Calculate onshore wind generation for German TSOs](#5.2.1-Calculate-onshore-wind-generation-for-German-TSOs)
		* [5.2.2 Calculate aggregate wind capacity for germany (on + offshore)](#5.2.2-Calculate-aggregate-wind-capacity-for-germany-%28on-+-offshore%29)
		* [5.2.3 Aggregate German data from individual TSOs and calculate availabilities/profiles](#5.2.3-Aggregate-German-data-from-individual-TSOs-and-calculate-availabilities/profiles)
	* [5.3 Create hourly data from 15' data](#5.3-Create-hourly-data-from-15'-data)
	* [5.4 Insert a column with Central European (Summer-)time](#5.4-Insert-a-column-with-Central-European-%28Summer-%29time)
	* [5.5 Reorder columns](#5.5-Reorder-columns)
* [6. Create metadata](#6.-Create-metadata)
* [7. Write data to disk](#7.-Write-data-to-disk)
	* [7.1 Different shapes](#7.1-Different-shapes)
	* [7.2 Write to SQL-database](#7.2-Write-to-SQL-database)
	* [7.3 Write to Excel](#7.3-Write-to-Excel)
	* [7.4 Write to CSV](#7.4-Write-to-CSV)
	* [7.5 Write checksums.txt](#7.5-Write-checksums.txt)


# 1. Introductory Notes

This Notebook handles missing data, performs calculations and aggragations and creates the output files.

# 2. Settings

## 2.1 Import Python libraries

This section: load libraries and set up a log.

Note that the download module makes use of the [pycountry](https://pypi.python.org/pypi/pycountry) library that is not part of Anaconda. Install it with with `pip install pycountry` from the command line.

In [ ]:
# Python modules
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import logging
import json
import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

# Skripts from time-series repository
from timeseries_scripts.read import read
from timeseries_scripts.download import download
from timeseries_scripts.imputation import find_nan
from timeseries_scripts.imputation import resample_markers
from timeseries_scripts.make_json import make_json

# Reload modules with execution of any code, to avoid having to restart
# the kernel after editing timeseries_scripts
%load_ext autoreload
%autoreload 2

sources_yaml_path = 'input/sources.yml'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('log')
# For more detailed logging messages, replace 'INFO' with 'DEBUG'
# (May slow down computation).
#logger.setLevel('INFO')

## 2.2 Set version number and recent changes

Executing this script till the end will create a new version of the data package.
The Version number specifies the local directory for the data <br>
We include a note on what has been changed.

In [ ]:
version = '2017-03-26'
data_path = os.path.join(version, 'original_data')
changes = '''include ENTSO E transparency'''

## 2.3 Select timerange

This section: select the time range and the data sources for download and read. Default: all data sources implemented, full time range available.

**Source parameters** are specified in [input/sources.yml](input/sources.yml), which describes, for each source, the datasets (such as wind and solar generation) alongside all the parameters necessary to execute the downloads.

The option to perform downloading and reading of subsets is for testing only. To be able to run the script succesfully until the end, all sources have to be included, or otherwise the script will run into errors (i.e. the step where aggregate German timeseries are caculated requires data from all four German TSOs to be loaded).

In order to do this, specify the beginning and end of the interval for which to attempt the download.

Type `None` to download all available data.

In [ ]:
start_from_user = None  # i.e. date(2016, 1, 1)
end_from_user = date(2016, 12, 31)  # i.e. date(2016, 1, 31)

## 2.4 Select download source

Instead of downloading from the sources, the complete raw data can be downloaded as a zip file from the OPSD Server. Advantages are:
- much faster download
- back up of raw data in case it is deleted from the server at the original source

In order to do this, specify an archive version to use the raw data from that version that has been cached on the OPSD server as input. All data from that version will be downloaded - timerange and subset will be ignored.

Type `None` to download directly from the original sources.

In [ ]:
archive_version = None  # i.e. '2016-07-14'

## 2.5 Select subset

Optionally, specify a subset to download/read.<br>
The next cell prints the available sources and datasets.<br>

In [ ]:
with open(sources_yaml_path, 'r') as f:
    sources = yaml.load(f.read())
for k, v in sources.items():
    print(yaml.dump({k: list(v.keys())}, default_flow_style=False))

Copy from its output and paste to following cell to get the right format.<br>
Type `subset = None` to include all data.

In [ ]:
subset = yaml.load('''
insert_source_here:
- insert_dataset1_from_that_source_here
- insert_dataset2_here
more_sources:
- more_data_sets
''')  
subset = None

Now eliminate sources and variables not in subset.

In [ ]:
with open(sources_yaml_path, 'r') as f:
    sources = yaml.load(f.read())
if subset:  # eliminate sources and variables not in subset
    sources = {source_name: {k: v
                             for k, v in sources[source_name].items()
                             if k in variable_list}
               for source_name, variable_list in subset.items()}

# 3. Download

This section: download data. Takes about 1 hour to run for the complete data set (`subset=None`).

First, a data directory is created on your local computer. Then, download parameters for each data source are defined, including the URL. These parameters are then turned into a YAML-string. Finally, the download is executed file by file.

Each file is saved under it's original filename. Note that the original file names are often not self-explanatory (called "data" or "January"). The files content is revealed by its place in the directory structure.

## 3.1 Automatic download (for all sources except Energinet.dk)

In [ ]:
download(sources, data_path,
         archive_version=archive_version,
         start_from_user=start_from_user,
         end_from_user=end_from_user)

## 3.2 Manual download (Energinet.dk)

Energinet.dk data needs to be downloaded manually from http://www.energinet.dk/en/el/engrosmarked/udtraek-af-markedsdata/Sider/default.aspx<br>
Check The Boxes as specified below, then press the "Get extract"-button at the end of the page.

You will receive a file `Market Data.xls` of about 50 MB. Open the file in Excel. There will be a warning from Excel saying that file extension and content are in conflict. Select "open anyways" and and save the file as `.xlsx`. This will compress the file by ~80%, resulting in faster processing afterwards. 

In order to be found by the read-function, place the downloaded file in the following subdirectory:<br>
` \original_data\Energinet.dk\prices_wind_solar\2000-01-01_2016-12-31\ ` <br>

**Boxes to check:**

Period<br>
Get data from: 01-01-2000 To: Today<br>
all months

Data columns <br>
Elspot Price, Currency Code/MWh

- DK-West
- DK-East
- Norway
- Sweden (SE)
- Sweden (SE3)
- Sweden (SE4)
- DE European Power Exchange

Production and consumption, MWh/h

- DK-West: Wind power production
- DK-West: Solar cell production (estimated)
- DK-East: Wind power production
- DK-East: Solar cell production (estimated)
- DK: Wind power production (onshore)
- DK: Wind power production (offshore)

Data format:<br>
Currency code
- EUR

Decimal format
- English number Format (period as decimal separator)

Date format
- Other date format(YYYY-MM-DD)

Recieve to
- Excel




# 4. Read

This section: Read each downloaded file into a pandas-DataFrame and merge data from different sources if it has the same time resolution. Takes ~15 minutes to run.

## 4.1 Preparations

Set the title of the rows at the top of the data used to store metadata internally. The order of this list determines the order of the levels in the resulting output.

In [ ]:
headers = ['region', 'variable', 'attribute', 'source', 'web']

Create a dictionary of empty DataFrames to be populated by the read function

In [ ]:
data_sets = {'15min': pd.DataFrame(), '60min': pd.DataFrame()}

## 4.2 Reading loop

Loop through sources and variables to do the reading

In [ ]:
# For each source in the source dictionary
for source_name, source_dict in sources.items():
    # For each variable from source_name
    for variable_name, param_dict in source_dict.items():
        variable_dir = os.path.join(data_path, source_name, variable_name)
        res_key = param_dict['resolution']
        url = param_dict['web']
        df = read(source_name, variable_name, url, res_key, headers,
                  out_path=data_path,
                  start_from_user=start_from_user,
                  end_from_user=end_from_user)

        if data_sets[res_key].empty:
            data_sets[res_key] = df
        elif not df.empty:
            data_sets[res_key] = (
                data_sets[res_key]
            ).combine_first(df)

Display some rows of the dataframes to get a first impression of the data.

In [ ]:
data_sets['15min']['2015-07-01 12:00':].tail()

In [ ]:
data_sets['60min']['2015-07-01 12:00':].tail()

## 4.3 Save raw data

Save the DataFrames created by the read function to disk. This way you have the raw data to fall back to if something goes wrong in the ramainder of this notebook without having to repeat the previos steps.

In [ ]:
data_sets['15min'].to_pickle('raw_15.pickle')
data_sets['60min'].to_pickle('raw_60.pickle')

Load the DataFrames saved above

In [ ]:
data_sets = {}
data_sets['15min'] = pd.read_pickle('raw_15.pickle')
data_sets['60min'] = pd.read_pickle('raw_60.pickle')

# 5. Processing

This section: missing data handling, aggregation of sub-national to national data, aggregate 15'-data to 60'-resolution. Takes 30 minutes to run.

## 5.1 Missing Data Handling

Patch missing data. At this stage, only small gaps (up to 2 hours) are filled by linear interpolation. This catched most of the missing data due to daylight savings time transitions, while leaving bigger gaps untouched

The exact locations of missing data are stored in the `nan_table` DataFrames.

Where data has been interpolated, it is marked in a new column `comment`. For eaxample the comment `solar_DE-transnetbw_generation;` means that in the original data, there is a gap in the solar generation timeseries from TransnetBW in the time period where the marker appears.

Patch the datasets and display the location of missing Data in the original data. Takes ~5 minutes to run.

In [ ]:
#data_sets['15min'], nan_table15 = find_nan(
#    data_sets['15min'], headers, patch=True)
data_sets['60min'], nan_table60 = find_nan(
    data_sets['60min'], headers, patch=True)

Execute this to see an example of where the data has been patched.

In [ ]:
data_sets['15min'][data_sets['15min']['interpolated_values'].notnull()].tail()

In [ ]:
data_sets['60min'][data_sets['60min']['interpolated_values'].notnull()].tail()

Display the table of regions of missing values

In [ ]:
nan_table15

In [ ]:
nan_table60

You can export the NaN-tables to Excel in order to inspect where there are NaNs

In [ ]:
writer = pd.ExcelWriter('NaN_table.xlsx')
nan_table15.to_excel(writer, '15min')
nan_table60.to_excel(writer, '60min')
writer.save()

Save/Load the patched data sets

In [ ]:
data_sets['15min'].to_pickle('patched_15.pickle')
data_sets['60min'].to_pickle('patched_60.pickle')

In [ ]:
data_sets = {}
data_sets['15min'] = pd.read_pickle('patched_15.pickle')
data_sets['60min'] = pd.read_pickle('patched_60.pickle')

## 5.2 Country specific calculations

### 5.2.1 Calculate onshore wind generation for German TSOs

For 50 Hertz, it is already in the data.
For TenneT, it is calculated by substracting offshore from total generation.
For Amprion and TransnetBW, onshore wind generation is just total wind generation.
Takes <1 second to run.

In [ ]:
# Some of the following operations require the Dataframes to be lexsorted in
# the columns
for res_key, df in data_sets.items():
    df.sortlevel(axis=1, inplace=True)

In [ ]:
for tso in ['DE_amprion', 'DE_tennet', 'DE_transnetbw']:
    new_col_header = {
        'variable': 'wind_onshore',
        'region': '{tso}',
        'attribute': 'generation',
        'source': 'own calculation',
        'web': ''
    }

    if tso == 'DE_tennet':
        colname = ('DE_tennet', 'wind_offshore', 'generation')
        offshore = data_sets['15min'].loc[:, colname]
    else:
        offshore = 0

    data_sets['15min'][
        tuple(new_col_header[level].format(tso=tso) for level in headers)
    ] = (data_sets['15min'][(tso, 'wind', 'generation')] - offshore)

    # Sort again
    data_sets['15min'].sortlevel(axis=1, inplace=True)

### 5.2.2 Calculate aggregate wind capacity for germany (on + offshore)

Apart from being interesting on it's own, this is also required to calculate an aggregated wind-profile for Germany

In [ ]:
new_col_header = {
    'variable': 'wind',
    'region': 'DE',
    'attribute': 'capacity',
    'source': 'BNetzA and Netztransparenz.de',
    'web': 'http://data.open-power-system-data.org/renewable_power_plants'
}
new_col_header = tuple(new_col_header[level] for level in headers)

data_sets['15min'][new_col_header] = (
    data_sets['15min']
    .loc[:, ('DE', ['wind_onshore', 'wind_offshore'], 'capacity')]
    .sum(axis=1, skipna=False))

# Sort again
data_sets['15min'].sortlevel(axis=1, inplace=True)

### 5.2.3 Aggregate German data from individual TSOs and calculate availabilities/profiles

The wind and solar in-feed data for the 4 German balancing areas is summed up and stored in in new columns, which are then used to calculate profiles, that is, the share of wind/solar capacity producing at a given time. The column headers are created in the fashion introduced in the read script. Takes 5 seconds to run.

In [ ]:
for tech in ['solar', 'wind', 'wind_onshore', 'wind_offshore']:
    for attribute in ['generation']:  # we could also include 'forecast'
        sum_col = data_sets['15min'].loc(axis=1)[(
            ['DE_50hertz', 'DE_amprion', 'DE_tennet', 'DE_transnetbw'],
            tech, attribute)].sum(axis=1, skipna=False).to_frame()

        # Create a new MultiIndex and append aggregated data to the dataset
        new_col_header = {
            'variable': '{tech}',
            'region': 'DE',
            'attribute': '{attribute}',
            'source': 'own calculation',
            'web': ''
        }
        tuples = [tuple(new_col_header[level].format(
            tech=tech, attribute=attribute) for level in headers)]
        sum_col.columns = pd.MultiIndex.from_tuples(tuples, names=headers)
        data_sets['15min'] = data_sets['15min'].combine_first(sum_col)

        if not attribute == 'generation':
            continue
        
        # Calculate the profile column
        profile_col = (sum_col.values /
                       data_sets['15min']['DE', tech, 'capacity'])
        
        # Create a new MultiIndex and append aggregated data to the dataset
        new_col_header = {
            'variable': '{tech}',
            'region': 'DE',
            'attribute': 'profile',
            'source': 'own calculation',
            'web': ''
        }
        tuples = [tuple(new_col_header[level].format(tech=tech)
                        for level in headers)]
        profile_col.columns = pd.MultiIndex.from_tuples(tuples, names=headers)
        data_sets['15min'] = data_sets['15min'].combine_first(profile_col)

## 5.3 Create hourly data from 15' data

Some data comes in 15-minute intervals (i.e. German renewable generation), other in 60-minutes (i.e. load data from ENTSO-E and Prices). We resample the 15-minute data to hourly resolution and append it to the 60-minutes dataset.

The marker column is resampled separately in such a way that all information on where data has been interpolated is preserved.

The `.resample('H').mean()` methods calculates the means from the values for 4 quarter hours [:00, :15, :30, :45] of an hour values, inserts that for :00 and drops the other 3 entries. Takes 15 seconds to run.

In [ ]:
# Condense marker column from 15 to 60 minutes resoltution
marker_15 = data_sets['15min']['interpolated_values'].groupby(
    pd.Grouper(freq='60Min', closed='left', label='left')
    ).agg(resample_markers)
marker_15 = marker_15.reindex(data_sets['60min'].index)

# Glue condensed 15 min marker onto 60 min marker
marker_60 = data_sets['60min']['interpolated_values']
both_markers = marker_15.notnull() & marker_60.notnull()
only_15 = marker_15.notnull() & marker_60.isnull()
data_sets['60min'].loc[both_markers, 'interpolated_values'] = (
    marker_60.str.cat(others=marker_15, sep=' | '))
data_sets['60min'].loc[only_15, 'interpolated_values'] = marker_15

In [ ]:
resampled = data_sets['15min'].resample('H').mean()
#try:
data_sets['60min'] = data_sets['60min'].combine_first(resampled)
#except KeyError:
#    data_sets['60min'] = resampled

## 5.4 Insert a column with Central European (Summer-)time

The index column of th data sets defines the start of the timeperiod represented by each row of that data set in **UTC** time. We include an additional column for the **CE(S)T** Central European (Summer-) Time, as this might help aligning the output data with other data sources.

In [ ]:
info_cols = {'utc': 'utc_timestamp',
             'cet': 'cet_cest_timestamp',
             'marker': 'interpolated_values'}

In [ ]:
for res_key, df in data_sets.items():
    if df.empty:
        continue
    df.index.rename(info_cols['utc'], inplace=True)
    df.insert(0, info_cols['cet'],
              df.index.tz_localize('UTC').tz_convert('Europe/Brussels'))

## 5.5 Reorder columns

Move the additional timestamp and the marker column to the left of the DataFrames

In [ ]:
# Not necessary for now
#for res_key, df in data_sets.items():
#    cols = df.columns.tolist()
#    cols = [cols[-1]] + [cols[-2]]+ cols[:-2]
#    data_sets[res_key] = df[cols]

# 6. Create metadata

This section: create the metadata, both general and column-specific. All metadata we be stored as a JSON file. Takes <1s to run.

In [ ]:
make_json(data_sets, info_cols, version, changes, headers)

# 7. Write data to disk

This section: Save as [Data Package](http://data.okfn.org/doc/tabular-data-package) (data in CSV, metadata in JSON file). All files are saved in the directory of this notebook. Alternative file formats (SQL, XLSX) are also exported. Takes about 1 hour to run.

But first, create a final savepoint

In [ ]:
data_sets['15min'].to_pickle('final_15.pickle')
data_sets['60min'].to_pickle('final_60.pickle')

In [ ]:
data_sets = {}
data_sets['15min'] = pd.read_pickle('final_15.pickle')
data_sets['60min'] = pd.read_pickle('final_60.pickle')

## 7.1 Different shapes

Data are provided in three different "shapes": 
- SingleIndex (easy to read for humans, compatible with datapackage standard, small file size)
  - Fileformat: CSV, SQLite
- MultiIndex (easy to read into GAMS, not compatible with datapackage standard, small file size)
  - Fileformat: CSV, Excel
- Stacked (compatible with data package standard, large file size, many rows, too many for Excel) 
  - Fileformat: CSV

The different shapes need to be created internally befor they can be saved to files. Takes about 1 minute to run.

In [ ]:
data_sets_singleindex = {}
data_sets_multiindex = {}
data_sets_stacked = {}
for res_key, df in data_sets.items():
    if df.empty:
        continue

    for col_name, col in df.iteritems():
        if not (col_name[0] in info_cols.values() or
                col_name[2] == 'profile'):
            df[col_name] = col.round(0)

    # MultIndex
    data_sets_multiindex[res_key + '_multiindex'] = df

    # SingleIndex
    df_singleindex = df.copy()
    # use first 3 levels of multiindex to create singleindex
    df_singleindex.columns = [
        col[0] if col[0] in info_cols.values()
        else '_'.join([level for level in col[0:3] if not level == ''])
        for col in df.columns.values]

    data_sets_singleindex[res_key + '_singleindex'] = df_singleindex

    # Stacked
    stacked = df.copy()
    stacked.drop(info_cols['cet'], axis=1, inplace=True)
    stacked.columns = stacked.columns.droplevel(['source', 'web'])
    stacked = stacked.transpose().stack(dropna=True).to_frame(name='data')
    data_sets_stacked[res_key + '_stacked'] = stacked

## 7.2 Write to SQL-database

This file format is required for the filtering function on the OPSD website. This takes about 30 seconds to complete.

In [ ]:
for res_key, df in data_sets_singleindex.items():
    table = 'time_series_' + res_key
    df = df.copy()
    df.index = df.index.strftime('%Y-%m-%dT%H:%M:%SZ')
    df[info_cols['cet']] = (df[info_cols['cet']]
                            .dt.strftime('%Y-%m-%dT%H:%M:%S%z'))
    filepath = os.path.join(version, 'time_series.sqlite')
    df.to_sql(table, sqlite3.connect(filepath),
              if_exists='replace', index_label=info_cols['utc'])

## 7.3 Write to Excel

Writing the full tables to Excel takes extremely long. As a workaround, only the first 5 rows are exported. The rest of the data can than be inserted manually from the `_multindex.csv` files.

In [ ]:
filepath = os.path.join(version, 'time_series.xlsx')
writer = pd.ExcelWriter(filepath)
for res_key, df in data_sets_multiindex.items():
    df.head().to_excel(writer, res_key, float_format='%.2f', merge_cells=True)
writer.save()

## 7.4 Write to CSV

This takes about 10 minutes to complete.

In [ ]:
# itertoools.chain() allows iterating over multiple dicts at once
for res_stacking_key, df in itertools.chain(
        data_sets_singleindex.items(),
        data_sets_multiindex.items(),
        data_sets_stacked.items()
    ):
    # convert the format of the cet_cest-timestamp to ISO-8601
    if not (res_stacking_key in ['15min_stacked', '60min_stacked']
            or type(df.iloc[0,0]) == str):
        df.iloc[:,0] = df.iloc[:,0].dt.strftime('%Y-%m-%dT%H:%M:%S%z')
    filepath = os.path.join(version, 'time_series_' + res_stacking_key + '.csv')
    df.to_csv(filepath, float_format='%.2f',
              date_format='%Y-%m-%dT%H:%M:%SZ')

## 7.5 Write checksums.txt

We publish SHA-checksums for the outputfiles on GitHub to allow verifying the integrity of outputfiles on the OPSD server.

In [ ]:
def get_sha_hash(path, blocksize=65536):
    sha_hasher = hashlib.sha256()
    with open(path, 'rb') as f:
        buffer = f.read(blocksize)
        while len(buffer) > 0:
            sha_hasher.update(buffer)
            buffer = f.read(blocksize)
        return sha_hasher.hexdigest()


filepath = os.path.join(version, 'checksums.txt')
files = os.listdir(version)

# Create checksums.txt in the version directory
with open(filepath, 'w') as f:
    for file_name in files:
        file_hash = get_sha_hash(os.path.join(version, file_name))
        f.write('{},{}\n'.format(file_name, file_hash))

# Copy the file to root directory from where it will be pushed to GitHub
copyfile(filepath, 'checksums.txt')